Phase Equilibrium Modelling
===========================



## Introduction



XRF (+titration) compositional data was collected by Peter Lindquist. Major element oxide and iron oxidation state data is contained within this data.



## Reading the Data



The raw data is stored in a .xslx Excel file, which is not easy to process in Python as a .csv file or Pandas DataFrame. As such, parsing the data into an easier format is the first step.

-   **Note: `openpyxl` (required for .xslx parsing) must be installed for this notebook to work**.

Units are wt% for the oxides, and ppm for the trace elements.

With there being 5 specimens, whose data is arranged columnwise, and an extra column containing row names, everything after the 6th column can be ignored. The first column contains the component being analysed, which is moved out of the dataframe structure in preparation for a transpose (such that each sample is represented by one row of data).



In [1]:
import pandas as pd
import numpy as np
import os

# Read raw data.
# There's an empty row in the raw data so set header row to second row (index: 1).
df = pd.read_excel(os.path.join("..","DATASETS","XRF","Serp-P.Lindquist. U.Wash. 11-2023.xlsx"),header=1)

# Ignore columns after the 6th.
df = df.iloc[:,:6]

# Extract component names.
components = list(df.iloc[:,0])
# Remove component names column from dataframe.
df.drop(df.columns[0],axis=1,inplace=True)

# Transpose df and add component names back in as column headers.
df = df.T
df.columns = components

# Save parsed df.
df.to_csv(os.path.join("local_data","df.csv"))

None

## Processing the Data for PEM



### Isolating Relevant Columns



For PEM, only certain element columns are of interest. Fe2O3T from the XRF analysis will be ignored in favor of FeO and Fe2O3 from the titration analysis. This also means the total from this raw data cannot be used. Water will be assumed to be the only volatile (LOI) for simplicity. A dehyrated total is also computed.



In [1]:
# State columns of interest.
columns_for_PEM = [c for c in df.columns if (len(c)>2) and (c not in ["Total","Fe2O3T"])]
# Extract columns of interest.
PEM_df = df[columns_for_PEM]
# Rename LOI column to H2O column.
PEM_df = PEM_df.rename(columns={"LOI":"H2O"})

# Compute dehydrated total.
dehyd_tot = PEM_df.drop("H2O",axis=1).sum(axis=1)
PEM_df["dehyd_tot"] = dehyd_tot

### Normalization with Observed H2O (LOI)



Since the XRF analysis looked at bulk oxide weight% after devolatilization (with LOI being recorded prior), `dehyd_tot` is around 100, with `tot` being > 100. Since LOI **is** being considered in the PEM compositions, the data needs to be renormalized such that `tot` is 100. However, due to the later measurement of the bulk oxide weight%, the columns making up `dehyd_tot` must be normalized to 100 - H2O. Since the data's `dehyd_tot` does not exactly sum to 100, the operation is performed on oxide weights expressed as fractions of the data's `dehyd_tot`.

-   Therefore: normed oxide = (observed oxide / observed dehyd tot) $\times$ normed dehyd tot, where normed dehyd tot is (100 - H2O). Rearranging, this produces: normed oxide = observed oxide $\times$ (normed dehyd tot / observed dehyd tot).
-   Strictly speaking, FeO and Fe2O3 (from titration) were not found in the same run (XRF) as the other major element oxides, so this is all a slight oversimplification.



In [1]:
# Store dehyd tot from the data outside of df.
obs_dehyd_tot = PEM_df["dehyd_tot"]
# Store H2O outside of df.
H2O = PEM_df["H2O"]
# Compute normed dehyd tot
norm_dehyd_tot = 100 - H2O

# Apply the normalizing factor to all columns in the table.
PEM_df_hyd = PEM_df.mul((norm_dehyd_tot/obs_dehyd_tot),axis=0)
# Save the renormalized observations database to disk.
PEM_df_hyd.to_csv(os.path.join("local_data","cleaned_normalized_df.csv"))

# Put correct H2O values back in.
PEM_df_hyd = PEM_df_hyd.assign(H2O=H2O,tot=PEM_df["dehyd_tot"]+H2O)

PEM_df_hyd

SiO2      TiO2     Al2O3  ...     Fe2O3 dehyd_tot         tot
23C-06B  40.009632  0.033558   1.40105  ...  6.975197     83.84  116.094177
23C-06C  45.221885  0.008676  0.728825  ...  6.785818     86.41  113.180922
23C-07A  38.964058  0.043236  1.781313  ...  7.831116     86.01  113.456297
23C-07B  39.408441  0.025718  1.431631  ...  7.341818     85.25  114.194241
23C-M02  39.915478  0.008533  1.459074  ...  7.384313     84.92  114.604241
:
[5 rows x 14 columns]

The totals now all sum to 100 as expected, and the oxides have been proportionally scaled.



### Normalization with Mantle H2O



A dehydrated/mantle composition can also be computed, with initial (mantle) water content set to 0.004 wt% \citep{Azevedo2021}. This can be done by repeating the previous but with H2O all set to 0.004.



In [1]:
# Store dehyd tot from the data outside of df.
obs_dehyd_tot = PEM_df["dehyd_tot"]
# Set H2O to 0.004.
H2O = 0.004
# Compute normed dehyd tot
norm_dehyd_tot = 100 - H2O

# Apply the normalizing factor to all columns in the table.
PEM_df_dehyd = PEM_df.mul((norm_dehyd_tot/obs_dehyd_tot),axis=0)

# Put correct H2O values back in.
PEM_df_dehyd = PEM_df_dehyd.assign(H2O=H2O,tot=PEM_df["dehyd_tot"]+H2O)

PEM_df_dehyd

SiO2      TiO2     Al2O3  ...     Fe2O3 dehyd_tot        tot
23C-06B  47.719503  0.040025  1.671033  ...   8.31932    99.996  99.938177
23C-06C  52.331994  0.010041  0.843417  ...  7.852733    99.996  99.594922
23C-07A  45.299965  0.050266   2.07097  ...  9.104526    99.996  99.470297
23C-07B  46.225061  0.030166  1.679266  ...  8.611759    99.996  99.448241
23C-M02  47.001744  0.010047  1.718106  ...  8.695263    99.996  99.528241
:
[5 rows x 14 columns]

### Constructing PEM Composition Strings



PEM composition strings for Theriak-Domino (T-D) are in the format X(Nx)Y(Ny) where X and Y are elements and Nx and Ny are their molar abundances. As such, converting from XRF data to T-D composition strings involves separating oxides into their constituent elements and then converting from mass (weight%) to moles (as molar ratios). The method is encoded in the spreadsheet `Composition-Converter-Palin.xlsx` (by Richard Palin), and translated to Python in the class `CompositionProcessor` in `composition_processor.py`. The specifics of the code are not relevant to this notebook, but a brief outline is provided for context (for more detail, see `composition_processor.py`, which contains explanatory comments). The general steps of this method include, where the moles are relative to each other rather than being absolute values (since the composition is in the form of percentages):

1.  Convert oxide weight to oxide moles through dividing by Mr.
2.  Convert all Fe2O3 into FeO + O (such that it's nominally stored in FeO, but with additional O stored separately that oxidizes the necessary fraction of FeO to Fe2O3).
3.  If an apatite correction is to be applied (i.e. remove apatite contributions to calcium oxide), then remove a number of CaO moles scaled to the amount of P2O5 observed. P2O5 can also be ignored by passing another option.
4.  (Not absolutely necessary but useful for inspection of oxide amounts) normalize the total number of oxide moles to 100 (such that the moles of each oxide effectively represents a percentage).
5.  Find the number of non-oxygen element atoms and oxygen atoms per oxide molecule and then multiply by the *oxide* moles to get the moles of each *element* (including oxygen after summing contributions from all oxides).
6.  Convert this data into a suitable format for T-D input.

An brief example of relevant usage is as follows:



In [1]:
from composition_processor import CompositionProcessor

# Access the (dehydrated) composition of the first sample in the database after removing the totals.
composition = dict(PEM_df_dehyd.iloc[1,:].drop(["tot","dehyd_tot"],axis=0))

print(composition)

td_formula = CompositionProcessor().theriak_domino_formula(composition)

print(td_formula)

{'SiO2': 52.33199408408726, 'TiO2': 0.010040674229487196, 'Al2O3': 0.8434166352769245, 'MnO': 0.15061011344230793, 'MgO': 37.86338251939622, 'CaO': 0.08032539383589757, 'Na2O': 0.040162696917948784, 'K2O': 0.010040674229487196, 'P2O5': 0.010040674229487196, 'H2O': 0.004, 'FeO': 0.8032539383589757, 'Fe2O3': 7.852732595995978}
SI(43.95)AL(0.83)CA(0.06)MG(47.40)FE(5.53)K(0.01)NA(0.07)TI(0.01)MN(0.11)H(0.02)O(144.79)

## PEM P-T-x Paths



The P-T-x path was determined (with some degree of interpretation and simplification) from the literature \citep{Grove1995,Platt2024}:

-   Cooling Path: 14 kbar, 850 deg C to 14 kbar, 700 deg C
    -   Composition: dehydrated/mantle water content
-   Serpentinisation and Exhumation Path: 14 kbar, 700 deg C to 4 kbar, 200 deg C
    -   Composition: hydrating (rapid increase to near observed water content at the start, and then slower increase to observed water content later in this path)
-   Final Exhumation Path: 4 kbar, 200 deg C to 1 kbar, 100 deg C
    -   Composition: hydrated/observed water content



In [1]:
import matplotlib.pyplot as plt

# Define the paths.
paths = {"cooling":([850,700],[14000,14000]),
         "serpentinisation":([700,200],[14000,4000]),
         "exhumation":([200,100],[4000,1000])}

plt.figure()
# Plot the paths.
for path in paths:
    plt.plot(*paths[path],linewidth=1.5,label=path)

# Set viewport limits.
plt.xlim(0,850)
plt.ylim(14100,0)
# Set axes labels.
plt.xlabel(r"Temperature /$^{\circ}\text{C}$")
plt.ylabel("Pressure /bar")

plt.legend()
plt.show()

None

## PEM Execution



The Python code used to interface with `theriak.exe`, and basic plotting methods for its output are not particularly relevant for the purposes of this notebook, and so are stored in the separate file `theriak_api.py`. This commented code file can be inspected in case of interest.

The following code imports the functions and classes from that file into this notebook session, where the purpse of each function/class method call will be clarified with comments.



In [1]:
from theriak_api import TheriakAPI,group_cols,TheriakOutput,read_theriak_table

# Imported:
# TheriakAPI (class) - handling the input for theriak.exe, including the construction of command/directive files.
# group_cols (function) - groups columns in a dataframe together into broader classifications (by default, this is applied to phases e.g. grouping fayalite and forsterite into olivine).
# TheriakOutput (class) - visualize the parsed output (dataframe) from theriak.exe using various plotting methods.

## Compositional Corrections



MnO can be removed from the compositions as it is not relevant for PEM. As a check of the compositions' suitability for PEM, the protolith mineralogy can be checked against expected mantle protolith mineralogy.



### Protolith Mineralogy with Compositions As-Is



This protolith mineralogy can be found by running `theriak.exe` for each composition (dehydrated/mantle composition) at the start of the serpentinisation path.



In [1]:
import shutil
import os

# Use theriak to regenerate data or read existing data produced by previous runs.
force_theriak_rerun = False

def find_protoliths(compositions_df,table_file_prepend=""):
    ''' Find the protolith of all samples in a composition dataframe, returning a list of theriak output tables parsed into pandas DataFrames and storing the output tables of each sample separately in raw output format.

    compositions_df | :pandas.DataFrame: | Compositions dataframe with row-wise samples.
    table_file_prepend | :str: | How to label the output table save files.

    Returns: :list: [:pandas.DataFrame:]
    '''
    # Extract the protolith P-T from serpentinisation path.
    PT = np.array(paths["serpentinisation"])[:,:-1]
    # Initiate theriak input control class with the relevant folder path and file names.
    theriak_api = TheriakAPI(theriak_dir="theriak",
                             ptx_commandfile="path.txt",
                             directive_file="path.directive")
    # Create theriak directive file. This only needs to be run once in this case (thermodynamic database doesn't change).
    theriak_api.create_directive()
    # Get list of sample names.
    samples = compositions_df.index
    # Initialize storage for outputted phase dataframes.
    dfs = []
    # Iterate through samples.
    for sample in samples:
        # Construct T-D formula for the active sample.
        td_formula = CompositionProcessor().theriak_domino_formula(compositions_df.loc[sample])
        print(sample,td_formula)
        # Make sure there are no old PTX path commands.
        theriak_api.clear_PTX_commands()
        # Provide theriak command to compute the stable mineral assemblage for the composition td_formula at the singular P-T conditions of PT
        theriak_api.add_PTX_command(td_formula,*PT.T[0][::-1],1)
        # Save the command to nonvolatile storage as a theriak path file.
        theriak_api.save_PTX_commandfile()
        # Run theriak.exe on the existing commands and retrieve the output table.
        df = theriak_api.execute_theriak()
        # Move output table to a more permanent location.
        shutil.move(os.path.join(theriak_api.theriak_dir,"loop_table"),
                    os.path.join("local_data","PEM",f"{table_file_prepend}-{sample}-loop_table"))
        # Store df.
        dfs.append(df)
    return dfs

# Clean composition dataframe (notably removing MnO).
dehyd_compositions_df = PEM_df_dehyd.drop(["tot","dehyd_tot","MnO"],axis=1)

# Get list of samples from compositions df.
samples = dehyd_compositions_df.index

# Declare the purpose of this PEM run.
table_file_prepend = "protoliths-unmodified"

if force_theriak_rerun:
    # Regenerate data if theriak is to be rerun.
    dfs = find_protoliths(dehyd_compositions_df,table_file_prepend=table_file_prepend)
else:
    # Otherwise read data produced by the previous run.
    dfs = [read_theriak_table(os.path.join("local_data","PEM",f"{table_file_prepend}-{sample}-loop_table")) for sample in samples]

To check against the expected mantle mineralogy, the phases must first be grouped to get a volume fraction of clinopyroxene, orthopyroxene and olivine.



In [1]:
def extract_umafic_protoliths(dfs):
    ''' Find the *ultramafic* protoliths (i.e. normalized proportions of Ol, Opx and Cpx) for all samples within a combined theriak output table.

    dfs | :list: [:pandas.DataFrame:] | List of theriak output tables.

    Returns: :list: [:np.array:]
    '''
    # Initialize list to store ultramafic protoliths.
    protoliths = []
    # Iterate through each sample's theriak output table.
    for df in dfs:
        # Isolate the volume/mineralogy columns.
        theriak_output = TheriakOutput(df)
        vol_df = theriak_output.extract_volumes()
        # Group minerals into broader classifications.
        protolith = group_cols(vol_df).iloc[0]
        # Define the necessary and only minerals for the ultramafic protolith.
        required = ["Ol","Opx","Cpx"]
        # Extract volumes these minerals from the grouped volume columns.
        protolith_umafic = np.array([(protolith[phase] if phase in protolith else 0) for phase in required])
        # Normalize and then save these volumes into the list.
        protoliths.append(protolith_umafic/protolith_umafic.sum())
    return protoliths

# Extract protoliths for the PEM results from unmodified compositions.
protoliths = extract_umafic_protoliths(dfs)

These [cpx,opx,ol] points can then be plotted on a ternary and compared against the expected range for mantle rocks \citep{Neumann2004}.



In [1]:
import mpltern
import json

def plot_umafic_ternary_base():
    ''' Produce the base plot for a ternary ultramafic a protolith characterization plot.

    Returns: :matplotlib.axes.Axes:
    '''
    # Initialize ternary plot.
    ax = plt.subplot(projection="ternary")
    # Dunite
    ax.plot([0.9,0.9],[0.1,0],[0,0.1],color="grey")
    # Lherzolite
    ax.plot([0.4,0.4],[0.6,0],[0,0.6],color="grey")
    # Ol Websterite
    ax.plot([0.05,0.05],[0.85,0.05],[0.05,0.85],color="grey")
    # Harzburgite
    ax.plot([0.85,0.05],[0.05,0.85],[0.05,0.05],color="grey")
    # Wherlite
    ax.plot([0.85,0.05],[0.05,0.05],[0.05,0.85],color="grey")
    # Cpxite
    ax.plot([0.1,0],[0,0.1],[0.9,0.9],color="grey")
    # Opxite
    ax.plot([0.1,0],[0.9,0.9],[0,0.1],color="grey")
    # Vertex labels
    ax.set_tlabel("Ol")
    ax.set_llabel("OPX")
    ax.set_rlabel("CPX")
    return ax

def plot_umafic_ternary(umafic_compositions):
    ''' Plot samples from an ultramafic mineralogy df onto an ultramafic ternary plot (Ol, Opx, Cpx).

    umafic_compositions | :list: [:np.array:] | List of normalized ultramafic modal mineralogy in order array([Ol, Opx, Cpx]).

    Returns: :matplotlib.axes.Axes:
    '''
    # Create ultramafic protolith ternary plot base.
    ax = plot_umafic_ternary_base()
    # Load polygon definition for expected range of (MOR) mantle.
    with open(os.path.join("local_data","Neumann2004_expected_mantle.json")) as infile:
        NA_MOR = json.load(infile)
    # Plot polygon for expected range.
    ax.fill(*np.array(NA_MOR).T,fc="pink",alpha=0.8,zorder=-1,label="Expected range")
    # Plot text labels for relevant lithologies (to the expected-range polygon).
    ax.text(*[2,1,1],"Lherzolite",ha="center",va="center")
    ax.text(*[28,1,1],"Dunite",ha="center",va="center")
    ax.text(*[2,1.5,0.1],"Harzburgite",ha="center",va="center",rotation=60)
    # Cast list of mineralogy arrays to (2D) numpy array.
    umafic_compositions = np.array(umafic_compositions)
    # Plot each composition (row in array) onto the ternary plot as points with label for sample ID.
    for i,P in enumerate(umafic_compositions):
        ax.plot(*P,label=samples[i],marker="*",markersize=10)
    # Show legend for each point.
    ax.legend()
    return ax

plt.figure()
# Plot protoliths for PEM results from unmodified compositions.
plot_umafic_ternary(protoliths)
plt.show()

None

Preliminary PEM modelling with the compositions as-is returned unexpected results in the ultramafic/mantle protolith.

Further investigation (e.g. of sample 23C-06B) also reveals the presence of unexpected phases in the mantle, namely haematite (instead of magnetite).



In [1]:
i = 0
print(samples[i])
vols = group_cols(TheriakOutput(dfs[i]).extract_volumes())
print(vols)

23C-06B
         Cpx        Opx      Mica        Hem         Ol
0  12.034895  900.72897  1.572977  69.657255  526.96759

This suggests that the observed (iron) composition is likely more oxidized than the protolith composition. \cite{Canil1994} suggests that mantle Fe2O3 ranges from 0.1 to 0.4 wt%, which is used to correct the observed compositions for the composition along the mantle cooling path. However, implementing this mantle Fe oxidation constraint is not as simple as setting the weight% of Fe2O3 to 0.1 and then adjusting FeO wt% to compensate to ensure a sum to 100, as that would change the (relative) molar total of Fe atoms in addition to changing the oxidation state. Nor would it be possible to set the weight% of Fe2O3 to 0.1, then compute the weight% of FeO from (relative) molar Fe as that may result in the wt% of all components not summing to 100% (resulting the Fe2O3 wt% being changed post-normalization). A more robust way of expressing oxidation than wt% of an individual oxide component is through the use of $Fe^{3+}/Fe_{tot}$ fraction, $f_{Fe3}$, which permits weight% to vary without being affected by initial weight% values.



### Iron Correction



As such, there should exist a unique value of $f_{Fe3}$ for each sample which results in the wt% of Fe2O3 being 0.1 wt%. "Analytical" method to compute Fe2O3 wt% from a prescribed $f_{Fe3}$:

1.  For the composition of interest, compute (relative) moles from wt% (moles = wt%/Mr).
2.  Compute total moles of Fe (sum of moles of Fe3+ = 2 \* moles of Fe2O3 and Fe2+ = moles of FeO).
3.  Find the necessary moles of Fe3+ such that Fe3+/Fe<sub>tot</sub>=f<sub>Fe3</sub> (by rearranging for Fe3+).
4.  Find the necessary moles of Fe2+ such that Fe3+ + Fe2+ = Fe<sub>tot</sub> (i.e. no change in the amount of Fe relative to the rest of the composition).
5.  Compute corresponding (new) moles of Fe2O3 and FeO (moles of Fe2O3 = moles of Fe3+ / 2; moles of FeO = moles of Fe2+) and update the composition.
6.  Compute unnormalized "wt%" of each oxide component in the updated composition.
7.  Compute the actual wt% of the oxide components via normalization (all components should sum to 100 wt%), which will change the wt% of all components. The wt% of Fe2O3 here can be compared to the desired value.

In this method, no oxides (e.g. MnO) shouldn't be dropped at the start since it's an observation that affects the total wt%. They can, however, be dropped afterwards.



In [1]:
from composition_processor import Molecule,normalise_dict_vals

def apply_Fe3_fraction(composition_wt,f_Fe3):
    ''' Apply a f_Fe3+ fraction (moles Fe3+/moles FeTot) to a wt% composition database, modifying it.

    f_Fe3+ | :float: | Fe3+/FeTot fraction to apply. Takes values in [0,1].
    composition_wt | :dict:-like | Composition of the sample expressed in oxide wt%.

    Returns: :dict:
    '''
    # Check whether the fraction can be applied.
    if not "FeO" in composition_wt and "Fe2O3" in composition_wt:
        raise ValueError("Both FeO and Fe2O3 must be present as oxides in the composition for f_Fe3+ to be applicable.")
    # Compute moles of each oxide component after casting wt% composition into dict.
    mol = CompositionProcessor().get_moles(dict(composition_wt))
    # Compute total moles of Fe atoms as a sum of Fe2+ and Fe3+ ions.
    mol_Fe = mol["FeO"] + 2 * mol["Fe2O3"]
    # Find the necessary moles of Fe3+ to get the requested Fe3+/FeTot fraction.
    mol_Fe3_new = f_Fe3 * mol_Fe
    # Find the necessary moles of Fe2+ to maintain the same FeTot:other elements molar ratio.
    mol_Fe2_new = mol_Fe - mol_Fe3_new
    # Update the composition in moles.
    mol["FeO"] = mol_Fe2_new
    mol["Fe2O3"] = mol_Fe3_new/2
    # Express the composition in terms of wt.
    wts = {k:v*Molecule(k).Mr() for k,v in mol.items()}
    # Normalize to get closured wt%.
    wts = normalise_dict_vals(wts)
    return wts

# Produce oxide compositions df without any oxide columns dropped.
compositions = PEM_df_dehyd.drop(["tot","dehyd_tot"],axis=1)
# Provide a demonstration f_Fe3+.
f_Fe3 = 0.1
# Compute the oxide composition after applying f_Fe3+ (for the first sample in `compositions`).
modified_df = apply_Fe3_fraction(compositions.iloc[0],f_Fe3)

print(modified_df)

{'SiO2': 48.074082795642745, 'Al2O3': 1.6834497435253386, 'CaO': 0.16128859818206837, 'MgO': 41.03786769995002, 'Fe2O3': 0.9647071334264746, 'FeO': 7.81248033521858, 'K2O': 0.020161074772758546, 'Na2O': 0.05040268693189637, 'TiO2': 0.04032214954551709, 'MnO': 0.14112752340930984, 'H2O': 0.004029722008905215, 'P2O5': 0.010080537386379273}

Due to the non-unique nature of mapping normalized wt% to unnormalized wt%, it's not possible to invert this method. A grid-search of different $f_{Fe3}$ values can be employed to find a suitable value such that the final Fe2O3 wt% = 0.1 wt%. Since the suitable $f_{Fe3}$ value depends on the initial composition (e.g. initial FeO and Fe2O3 wt% values), it is not the same for all samples. Due to the monotonically increasing nature of the relation between $f_{Fe3}$ and Fe2O3 wt%, if a test $f_{Fe3}$ produces Fe2O3 wt% > 0.1, then $f_{Fe3}$ just needs to be reduced and vice versa. As such, a simple range-narrowing iterative algorithm can be produced to find the most-suitable $f_{Fe3}$.



In [1]:
def range_halving_convergence(func,target,x_range,tolerance=1e-5,max_iter=100):
    ''' Converge on a x value which results in func(x) ~ some target, with the level of approximation decided by a tolerance.

    func | :function: | Monotonic, function that takes a single numerical input ("x") and returns another number ("y"). Must be valid over `x_range`.
    target | :Numerical: | The y value which is to be fitted by func(x).
    x_range | [:Numerical:,:Numerical:] | The finite x range over which to search for the best-fit x value.
    tolerance | :Numerical: | The acceptable difference between func(x) and target before declaring a best-fit x value found.
    max_iter | :int: | The maximum number of range halvings before declaring a failure to find a within-tolerance match.

    Returns: :Numerical:
    '''
    # Start off with a very high misfit.
    misfit = 1e6
    # Initialize variable to accumulate the iteration count.
    i = 0
    # Continue the range halving algorithm as long as the maximum number of iterations isn't yet hit or a match has been found.
    while i < max_iter and misfit > tolerance:
        # Find the midpoint of the range.
        x = (x_range[1] + x_range[0])/2
        # Check the output ("y") of the function at the midpoint of the range.
        found = func(x)
        # Compute the misfit.
        misfit = abs(found - target)
        if found > target:
            # If this output y is larger than the target y, set the subsequent range to the lower half range [min,midpoint].
            x_range[1] = x
        else:
            # Otherwise, set the subsequent range to the upper half range [midpoint,max].
            x_range[0] = x
        # Increment the iteration counter.
        i += 1
    # Display whether a within-tolerance x value was found.
    if i == max_iter:
        print("No satisfactory convergence")
    else:
        print("Convergence found: func(%s) ~ %s" % (x,found))
    return x

Applying this method to all the compositions.



In [1]:
def find_Fe3_fractions(compositions,target_wt):
    ''' Find an acceptable f_Fe3 value for each sample that will ensure Fe2O3 wt% equals the target_wt %.

    compositions | :pandas.DataFrame: | XRF-related oxide composition dataframe with row-wise samples.
    target_wt | Numerical | Target wt% for Fe2O3.

    Returns: :dict: {"<Sample name>":<f_Fe3 value>}
    '''
    # Initialize dictionary to store found f_Fe3 values for different samples.
    f_Fe3_values = dict()
    # Iterate through samples.
    for sample in compositions.index:
        # Isolate data for each sample.
        composition = compositions.loc[sample]
        # Declare function that will map a Fe3+/FeTot fraction to Fe2O3 wt%.
        func = lambda fraction : apply_Fe3_fraction(composition,fraction)["Fe2O3"]
        # Search for a suitable Fe3+/FeTot fraction using the range halving function and accepting the default search options.
        f_Fe3 = range_halving_convergence(func,target_wt,[0,1])
        # Store the found f_Fe3 value.
        f_Fe3_values[sample] = f_Fe3
    return f_Fe3_values

# Declare target.
Fe2O3_target = 0.1 # wt% Fe2O3
# Find acceptable f_Fe3 values that ensure Fe2O3 wt% ~0.1 for the samples in the unmodified compositions df.
f_Fe3_values = find_Fe3_fractions(compositions,Fe2O3_target)

Convergence found: func(0.010356903076171875) ~ 0.10000042657380336
Convergence found: func(0.01134490966796875) ~ 0.09999282576990538
Convergence found: func(0.009136199951171875) ~ 0.10000685396866496
Convergence found: func(0.010356903076171875) ~ 0.09999761281927519
Convergence found: func(0.010267257690429688) ~ 0.09999724575574237

The tolerated $f_{Fe3}$ values for Fe2O3 wt% &asymp; 0.1 is near 0.01, but with some variation for the different samples (up to +13%). These $f_{Fe3}$ values can be used to correct the Fe oxidation state of observed compositions and then used to find protoliths again.



In [1]:
force_theriak_rerun = False

def correct_all_sample_compositions(compositions,application_function,corrections):
    ''' Update all samples in a wt% compositions dataframe (with row-wise samples) with a function that takes a samples composition and modifies it given a value or values.

    compositions | :pd.DataFrame: | Wt% compositions dataframe with row-wise samples.
    application_function | function | Function that takes the inputs: sample oxide composition and correction object, and then modifies the composition based on the contents/value of the correction object.
    corrections | :dict: {"<Sample name>":<correction object>} | Dictionary of correction objects suitable for input into application_function.
    '''
    # Iterate through samples in the compositions df.
    for sample in compositions.index:
        # Compute the corrected composition for the active sample.
        corrected_composition = application_function(compositions.loc[sample],corrections[sample])
        # Update the old composition with this corrected composition.
        compositions.loc[sample] = pd.Series(corrected_composition)
    return compositions

# Modify the compositions by applying f_Fe3 values that were found to bring Fe2O3 wt% to 0.1.
compositions = correct_all_sample_compositions(compositions,apply_Fe3_fraction,f_Fe3_values)

# Now remove MnO.
dehyd_compositions_df = compositions.drop(["MnO"],axis=1)

table_file_prepend = "protoliths-fe-corrected"
if force_theriak_rerun:
    # Regenerate all protoliths data by running theriak.
    dfs = find_protoliths(dehyd_compositions_df,table_file_prepend=table_file_prepend)
else:
    # Load all protoliths data from previous run.
    dfs = [read_theriak_table(os.path.join("local_data","PEM",f"{table_file_prepend}-{sample}-loop_table")) for sample in samples]

These protoliths can be loaded inspected on a ultramafic ternary plot again.



In [1]:
plt.figure()
# Find ultramafic protolith compositions.
fe_corr_protoliths = extract_umafic_protoliths(dfs)
# Plot ultramafic protoliths onto an Ol-Opx-Cpx ternary.
plot_umafic_ternary(fe_corr_protoliths)
plt.show()

None

The ultramafic protolith lithologies are starting to lie closer to the expected range (with some even lying *within* the expected range). However, the protoliths all appear a bit Ol-depleted (and pyroxene-enriched) compared to expected, which is a symptom of SiO2 enrichment.



### SiO2 Correction



The SiO2 enrichment of an originally more SiO2-depleted mantle protolith is supported by \cite{Bebout1989}, who found that the SiO2 added during serpentinisation. As such, the mantle SiO2 is to be reduced to the amount expected for mantle rocks - i.e. 44 wt% \cite{Benard2021}. Though setting SiO2 in all samples (without any oxide columns removed) to 44 wt% may appear to be the simple solution to this, this change will not only change the Fe2O3 wt% away from 0.1 wt% after normalization, but also modify give rise to a different SiO2 wt% after normalization too. To fix the second issue, a range halving convergence search can be performed for each sample.



In [1]:
def modify_SiO2(composition,new_SiO2):
    ''' Modify the SiO2 value in a dict-like composition for a single composition and then normalize the resulting composition.

    composition | :dict:-like | Wt% oxide composition for a sample.
    new_SiO2 | Numerical | SiO2 wt% to apply before renormalization to 100%.

    Returns: :dict:
    '''
    # Cast composition to dict.
    composition = dict(composition)
    # Update SiO2 wt%.
    composition["SiO2"] = new_SiO2
    # Normalize all wt% values to 100%.
    composition = normalise_dict_vals(composition)
    return composition

def find_SiO2_values(compositions,target_wt):
    ''' For all samples in a compositions df (row-wise samples), find suitable pre-renormalization SiO2 wt% values that will result in the desired SiO2 wt% value *after* renormalization.

    compositions | :pandas.DataFrame: | XRF-related oxide composition dataframe with row-wise samples.
    target_wt | Numerical | Target wt% for SiO2.

    Returns: :dict: {"<Sample name>":<SiO2 wt%>}
    '''
    # Initialize dictionary to store acceptable SiO2 wt% values.
    SiO2_values = dict()
    # Iterate through samples.
    for sample in compositions.index:
        # Isolate active sample's composition.
        composition = compositions.loc[sample]
        # Declare function that will map a pre-normalization SiO2 wt% to post-normalization SiO2 wt%.
        func = lambda SiO2 : modify_SiO2(composition,SiO2)["SiO2"]
        # Search for a suitable SiO2 wt% using the range halving function and accepting the default search options.
        SiO2 = range_halving_convergence(func,target_wt,[0,100],tolerance=0.01)
        # Store suitable SiO2 wt%.
        SiO2_values[sample] = SiO2
    return SiO2_values

# Declare target.
SiO2_target = 44 # wt% SiO2
# Find acceptable pre-normalization SiO2 wt% values that ensure post-normalization SiO2 wt% ~ 44.
SiO2_values = find_SiO2_values(compositions,SiO2_target)
# Modify the compositions by applying these SiO2 wt% values.
compositions = correct_all_sample_compositions(compositions,modify_SiO2,SiO2_values)
# Print the post SiO2 correction Fe2O3 wt% values.
print("Fe2O3 wt%%\n%s" % compositions["Fe2O3"])

#+begin_example
Convergence found: func(40.771484375) ~ 44.00320639441022
Convergence found: func(37.1337890625) ~ 44.001506657225605
Convergence found: func(42.6513671875) ~ 43.998167920753886
Convergence found: func(41.943359375) ~ 44.00237849997184
Convergence found: func(41.30859375) ~ 43.99279964122063
Fe2O3 wt%
23C-06B    0.107927
23C-06C    0.118486
23C-07A    0.103165
23C-07B    0.104907
23C-M02    0.106495
Name: Fe2O3, dtype: object
#+end_example

Although the SiO2 wt% is now close to 44, the Fe2O3 wt% has been modified up to +19% from 0.1 (for 06C). One way to tackle this issue would be to iteratively correct alternate oxides until the misfit on both is satisfactory.



In [1]:
# Declare finality conditions.
SiO2_tolerance = 0.01
Fe2O3_tolerance = 0.0005
max_iter = 20
# Initialize iteration counter.
i = 0
# Compute the differences between the oxides being actively corrected and their target values.
SiO2_diff = abs(compositions["SiO2"] - SiO2_target)
Fe2O3_diff = abs(compositions["Fe2O3"] - Fe2O3_target)
# Iterate as long as none of the finality conditions are not met.
while not (all(SiO2_diff<SiO2_tolerance) and all(Fe2O3_diff<Fe2O3_tolerance)) and i < max_iter:
    # Find acceptable f_Fe3 values that ensure Fe2O3 wt% ~0.1 for the samples in the unmodified compositions df.
    f_Fe3_values = find_Fe3_fractions(compositions,Fe2O3_target)
    # Modify the compositions by applying these f_Fe3 values.
    compositions = correct_all_sample_compositions(compositions,apply_Fe3_fraction,f_Fe3_values)
    # Find acceptable pre-normalization SiO2 wt% values that ensure post-normalization SiO2 wt% ~ 44.
    SiO2_values = find_SiO2_values(compositions,SiO2_target)
    # Modify the compositions by applying these SiO2 wt% values.
    compositions = correct_all_sample_compositions(compositions,modify_SiO2,SiO2_values)
    # Compute the differences between the oxides being actively corrected and their target values.
    SiO2_diff = abs(compositions["SiO2"] - SiO2_target)
    Fe2O3_diff = abs(compositions["Fe2O3"] - Fe2O3_target)
    # Increment iteration counter.
    i += 1

# Declare that acceptable compositions have been found if the composition-related finality conditions (conditions of acceptability) have been met before reaching the maximum number of iterations.
if i != max_iter:
    print("Acceptable compositions found")

The change in water wt% as a result of this process is ignored since it is very small.

With an acceptable composition found, protolith PEM can be rerun.



In [1]:
force_theriak_rerun = False

# Now remove MnO.
dehyd_compositions_df = compositions.drop(["MnO"],axis=1)

table_file_prepend = "protoliths-si-fe-corrected"
if force_theriak_rerun:
    # Regenerate data if theriak is to be rerun.
    dfs = find_protoliths(dehyd_compositions_df,table_file_prepend=table_file_prepend)
else:
    # Otherwise read data produced by the previous run.
    dfs = [read_theriak_table(os.path.join("local_data","PEM",f"{table_file_prepend}-{sample}-loop_table")) for sample in samples]

plt.figure()
# Find ultramafic protolith compositions.
fe_si_corr_protoliths = extract_umafic_protoliths(dfs)
# Plot ultramafic protoliths onto an Ol-Opx-Cpx ternary.
plot_umafic_ternary(fe_si_corr_protoliths)
plt.show()

None

With this silica correction added on, the ultramafic protoliths plot much closer to expected, with the samples lying in or very close to the expected range from \cite{Neumann2004}. As such, these updated compositions in `dehyd_compositions_df` are accepted as protolith compositions.



## Serpentinisation Path Corrections



Since the oxidation that affected $f_{Fe3}$ likely arose at least partially from being near the (oxidizing) earth's surface (i.e. is a recent effect), the final composition in the PEM (i.e. the final serpentinite composition) will also be Fe-oxidation corrected: \cite{Eberhard2023} finds that antigorite serpentinite has a Fe3+/FeTot is 0.4, which will be assumed true for the theoretical, unweathered serpentinite of Santa Catalina.

With water following a initially rapid then slower increase along the serpentinisation path \citep{Grove1995}, and silica being introduced by the water \citep{Bebout1989}, the serpentinisation path results in an increase in both water and silica proportional to each other (i.e. following the same relative path) before reaching the observed values (i.e. their addition is not assumed to be related to surface processes).

The changes along the serpentinisation path are summarized in Table [1](#orgb0b042c).


| Component|Protolith|End|
|---|---|---|
| H2O|0.004 wt%|observed|
| SiO2|44 wt%|observed|
| Fe oxidation|Fe2O3 = 0.1 wt%|Fe3/FeTot = 0.4|

The compositions of the post-serpentinisation rock require only a modification to their Fe oxidation state.



In [1]:
# Modify the non-dehydrated composition df (after removing non-oxide columns) by ensuring their f_Fe3 values are all 0.4.
hyd_compositions_df = correct_all_sample_compositions(PEM_df_hyd.drop(["dehyd_tot","tot"],axis=1),apply_Fe3_fraction,{sample:0.4 for sample in PEM_df_hyd.index})
# Remove MnO from compositions.
hyd_compositions_df.drop(["MnO"],axis=1,inplace=True)

None

## PEM Running



With the protolith and final compositions found (`dehyd_compositions_df` and `hyd_compositions_df` respectively), the full PEM can be constructed.

The number of steps along each path must be even and at least 6, and is declared by the variable n:



In [1]:
# Number of steps in each P-T path segment, must be even.
n = 8
if n%2 != 0:
    raise ValueError("n must be even")
if n < 6:
    raise ValueError("n must be at least 6")

### Cooling Path



The cooling path simply involves changing the P-T of modelling for a constant composition (protolith composition). The P-T range to be covered is retrieved from `paths["cooling"]` and interpolated along.



In [1]:
def PT_change_path(composition,TP_path,n_steps,table_file_prepend):
    ''' Generate a PTX path file for a linear PT path with a prescribed number of intermediate steps and execute theriak on that file.

    composition | :dict:-like {<Oxide>:<wt%>} | Oxide composition for one sample.
    TP_path | :list:-like [[T0,T1],[P0,P1]] | Linear PT-path defined by start and end T and P values. Note that T precedes P.
    n_steps | :int: | Number of steps to perform PEM at along the TP_path.
    table_file_prepend | :str: | How to label the output table save files.

    Returns: :pandas.DataFrame:
    '''
    # Initialize new TheriakAPI instance, accepting the default folder/file paths.
    theriak_api = TheriakAPI()
    # Convert the dict-like composition into a theriak-domino string composition.
    composition = CompositionProcessor().theriak_domino_formula(composition)
    # Generate commands for PEM on a linear PT path for the desired composition.
    theriak_api.add_PTX_command(composition,TP_path[1],TP_path[0],n_steps)
    # Write both the PTX commandfile and the directive file for running this path commandfile.
    theriak_api.save_all()
    # Execute theriak and retrieve the output table.
    df = theriak_api.execute_theriak()
    # Move the raw output table file to a more static location.
    shutil.move(os.path.join(theriak_api.theriak_dir,"loop_table"),
                    os.path.join("local_data","PEM",f"{table_file_prepend}-loop_table"))
    return df

def PT_change_path_all(compositions,TP_path,n_steps,table_file_prepend):
    ''' Perform a linear PT-path PEM for all samples in a compositions dataframe.

    compositions | :pandas.DataFrame: | Compositions dataframe with row-wise samples.
    TP_path | :list:-like [[T0,T1],[P0,P1]] | Linear PT-path defined by start and end T and P values. Note that T precedes P.
    n_steps | :int: | Number of steps to perform PEM at along the TP_path.
    table_file_prepend | :str: | How to label the output table save files.

    Returns: :list: [:pandas.DataFrame:]
    '''
    # Extract sample names.
    samples = compositions.index
    # Initialize list to hold output table dataframes.
    dfs = []
    # Iterate through samples.
    for sample in samples:
        # Isolate composition for active sample.
        composition = compositions.loc[sample]
        # Execute PEM for the specified PT path for active sample and retrieve output table.
        df = PT_change_path(composition,TP_path,n_steps,table_file_prepend+f"-{sample}")
        # Store output table.
        dfs.append(df)
    return dfs

def cooling_path_all(n_steps,table_file_prepend):
    ''' Perform a cooling-path PEM for all samples in the protolith (dehydrated) compositions dataframe.

    n_steps | :int: | Number of steps to perform PEM at along the TP_path.
    table_file_prepend | :str: | How to label the output table save files.

    Returns: :list: [:pandas.DataFrame:]
    '''
    return PT_change_path_all(dehyd_compositions_df,paths["cooling"],n_steps,table_file_prepend)

In [1]:
force_theriak_rerun = False

table_file_prepend = "cooling"
if force_theriak_rerun:
    # Regenerate data if theriak is to be rerun.
    dfs = cooling_path_all(n,table_file_prepend=table_file_prepend)
else:
    # Otherwise read data produced by the previous run.
    dfs = [read_theriak_table(os.path.join("local_data","PEM",f"{table_file_prepend}-{sample}-loop_table")) for sample in samples]

The first output can be checked for sensibility.



In [1]:
# Remove previous plots from cache (important when the org version is run).
plt.close("all")
# Produce plots to overview the PEM output for the first sample.
TheriakOutput(dfs[0]).characterize_output()
plt.show()

None

### Final Exhumation Path



The cooling path also involves changing the P-T of modelling for a constant composition (final composition). The P-T range to be covered is retrieved from `paths["exhumation"]` and interpolated along.



In [1]:
force_theriak_rerun = False

def exhumation_path_all(n_steps,table_file_prepend):
    ''' Perform an exhumation-path PEM for all samples in the final rock (hydrated) compositions dataframe.

    n_steps | :int: | Number of steps to perform PEM at along the TP_path.
    table_file_prepend | :str: | How to label the output table save files.

    Returns: :list: [:pandas.DataFrame:]
    '''
    return PT_change_path_all(hyd_compositions_df,paths["exhumation"],n_steps,table_file_prepend)

table_file_prepend = "exhumation"

if force_theriak_rerun:
    # Regenerate data if theriak is to be rerun.
    dfs = exhumation_path_all(n,table_file_prepend=table_file_prepend)
else:
    # Otherwise read data produced by the previous run.
    dfs = [read_theriak_table(os.path.join("local_data","PEM",f"{table_file_prepend}-{sample}-loop_table")) for sample in samples]

The first output can be checked for sensibility.



In [1]:
# Remove previous plots from cache (important when the org version is run).
plt.close("all")
# Produce plots to overview the PEM output for the first sample.
TheriakOutput(dfs[0]).characterize_output()
plt.show()

None

### Serpentinisation Path



The serpentinisation path is more complicated as it involved changing both the P-T and composition along the path. The P-T range to be covered is retrieved from `paths["serpentinisation"]`, and the compositions to be covered are between the protolith and final compositions. With 8 steps in the PEM, the composition will be shifted away from protolith (30%) to final composition (70%) in the first two steps, with the remaining shift towards final composition coming from the remaining six steps. This is the case when $PT_{0.7}$ is set to 30%. $PT_{0.7}$ can be varied.

-   Where $PT_{0.7}$ is the \textit{P-T} condition when $C_{in} = 0.7(C_{final} - C_{proto}) + C_{proto}$ (where $C$ is composition and the subscript denotes \textit{in}put, \textit{proto}lith (initial) and \textit{final})



In [1]:
PTpointseven = 30 / 100 # 30%

Intermediate compositions will be treated as linear mixtures between the two endpoint composition.



In [1]:
def mix_endmembers(endmember_1,endmember_2,frac_2):
    ''' Find the elemental composition that represents the mixture of two endmembers with a prescribed fraction of the second endmember. The elements present within endmember 2 must also be present in endmember 1 (but not strictly the opposite).

    endmember_1 | :dict: {"<Element>":<amount>} | Elemental composition dictionary for endmember 1.
    endmember_2 | :dict: {"<Element>":<amount>} | Elemental composition dictionary for endmember 2.
    frac_2 | :float: | Fraction of endmember 2 in the mixture.

    Returns: :dict: {"<Element>":<amount>}
    '''
    # Initialize output dict.
    out = endmember_1.copy()
    # Iterate through the elements in the composition dictionary of endmember 1.
    for elem in endmember_1:
        # Cast the amount of the active element from endmember 1's composition into a float.
        x1 = float(endmember_1[elem])
        if elem in endmember_2:
            # If the element is also present in endmember 2, cast the amount of the active element from endmember 2's composition into a float.
            x2 = float(endmember_2[elem])
        else:
            # Otherwise explicitly declare the amount of the active element in endmember 2 as zero.
            x2 = 0
        # Compute the amount of active element in the (linear) mixture and store the result in the output composition.
        out[elem] = (x1 * (1-frac_2) + x2 * (frac_2))
    return out

# Function to set the first three values in a list on n numbers to values increasing linearly from 0 to 0.7, with the remaining values increasing linearly to 1.
# Intended to represent the fraction of the post-serpentinisation composition in a mixture of the pre- and post-serpentinisation compositions.
interp_coords_f = lambda n : np.append(np.linspace(0,0.7,int(PTpointseven*n)),
                            np.linspace(0.7,1,n-(int(PTpointseven*n)-1))[1:],axis=0)

With intermediate compositions found, the PEM can be set up. However another issue with the serpentinisation path is the large number variables (columns in the output table) stored by theriak. This is due to theriak storing all history in one table. The way around this issue is to execute only one point and accumulate the loop table in Python (as a pandas dataframe).



In [1]:
force_theriak_rerun = False

def serpentinisation_path(sample,n_steps,table_file_prepend):
    ''' Perform a serpentinisation-path PEM (which involves compositional change) for a single sample.
    '''
    # Isolate protolith composition for sample of interest.
    composition_1 = dict(dehyd_compositions_df.loc[sample])
    # Isolate post-serpentinisation composition for sample of interest.
    composition_2 = dict(hyd_compositions_df.loc[sample])
    # Generate list of theriak-domino composition strings representing mixtures of protolith and post-serpentinisation compositions using mixture fractions generated by interp_coords_f().
    interpolated_compositions = [CompositionProcessor().theriak_domino_formula(mix_endmembers(composition_1,composition_2,f)) for f in interp_coords_f(n_steps)]
    # Generate list of interpolated PT points (in TP order) corresponding to the changing compositions.
    interpolated_TP = np.linspace(*np.array(paths["serpentinisation"]).T,n_steps)

    # Initialize new TheriakAPI instance, accepting the default folder/file paths.
    theriak_api = TheriakAPI()
    # Write directive file.
    theriak_api.create_directive()
    # Initialize list to store theriak output table.
    combined_df = []
    # Iterate through interpolated compositions and PT points.
    for composition,TP in zip(interpolated_compositions,interpolated_TP):
        # Ensure the PTX command set is empty.
        theriak_api.clear_PTX_commands()
        # Generate command for a single PT point PEM for the active composition.
        theriak_api.add_PTX_command(composition,TP[1],TP[0],1)
        # Write PTX commandfile.
        theriak_api.save_PTX_commandfile()
        # Execute theriak and retrieve the output table.
        df = theriak_api.execute_theriak()
        # Clean the column names.
        df.columns = [c.replace(" ","") for c in df.columns]
        # Store data.
        combined_df.append(df)

    # Combine stored data.
    combined_df = pd.concat(combined_df,axis=0).fillna(0)
    # Save the combined output table.
    combined_df.to_csv(os.path.join("local_data","PEM",f"{table_file_prepend}-{sample}-loop_table"),index=False)
    return combined_df

def serpentinisation_path_all(n_steps,table_file_prepend):
    ''' Run the serpentinisation path PEM for all samples (implicitly from the original, unmodified composition dataframe).

    n_steps | :int: | Number of steps to perform PEM at along the TP_path.
    table_file_prepend | :str: | How to label the output table save files.

    Returns: :list: [:pandas.DataFrame:]
    '''
    # # Extract sample names.
    samples = compositions.index
    # Initialize list to hold output table dataframes.
    dfs = []
    # Iterate through samples.
    for sample in samples:
        # Execute serpentinisation PEM for the active sample and retrieve the theriak output table (dataframe).
        df = serpentinisation_path(sample,n_steps,table_file_prepend)
        # Store the theriak output table.
        dfs.append(df)
    return dfs

table_file_prepend = "serpentinisation"
if force_theriak_rerun:
    # Regenerate data if theriak is to be rerun.
    dfs = serpentinisation_path_all(n,table_file_prepend=table_file_prepend)
else:
    # Otherwise read data produced by the previous run.
    dfs = [read_theriak_table(os.path.join("local_data","PEM",f"{table_file_prepend}-{sample}-loop_table")) for sample in samples]

The first output can be checked for sensibility.



In [1]:
# Remove previous plots from cache (important when the org version is run).
plt.close("all")
# Produce plots to overview the PEM output for the first sample.
TheriakOutput(dfs[0]).characterize_output()
plt.show()

None

### Postscript



To avoid creating and excessively long notebook, result visualization will be handled in a separate notebook (`results.org/results.ipynb`).



## Compositions Used for PEM



Post-correction (including post apatite correction) compositions can be found and shown from pre-apatite correction compositions:



In [1]:
col_order = ["SiO2","TiO2","Al2O3","MnO","MgO","CaO","Na2O","K2O","P2O5","H2O","FeO","Fe2O3","O"]

def apatite_corrected_compositions(composition_df,col_order=col_order):
    c = CompositionProcessor()
    apatite_corrected = []
    for _,row in composition_df.iterrows():
        c.load_composition(row.to_dict())
        apatite_corrected.append(c.get_standardised_oxides())
    apatite_corrected_df = pd.DataFrame(apatite_corrected)
    apatite_corrected_df.index = composition_df.index
    for col in col_order:
        if not col in apatite_corrected_df:
            apatite_corrected_df[col] = ""
    return apatite_corrected_df[col_order]

print(apatite_corrected_compositions(dehyd_compositions_df).round(5).to_latex())
print(apatite_corrected_compositions(hyd_compositions_df).round(5).to_latex())

None